<a href="https://colab.research.google.com/github/blauveltmr/Data-Analysis/blob/master/DetailedSignals_StrategyAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
start_date = '2015-01-01'
end_date = '2021-01-06'
date_fmt = '%Y-%m-%d'
etflist = ['agq','bib','bis','boil','brzu','bzq','drn','drv','dust','dwt','edc','edz','erx','ery','fas','faz','fngd','fngu','gll','jdst','jnug','jpnl','kold','labd','labu',
  'nugt','qid','qld','rusl','russ','sdow','soxl','soxs','spxl','spxs','svxy','tecl','tecs','tmf','tmv','tna','tza','udow','ugl','uvxy','uwt','yang','yinn','zsl']

init_balance = 15000.00

#etflist = ['labu']  
import time

tic = time.time()

In [ ]:
!pip install yfinance
!pip install ta

In [ ]:
import yfinance as yf
import ta
import pandas as pd
from datetime import date, timedelta, datetime
from IPython.display import clear_output

**Compute Buffer**



In [ ]:
start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(days=365)
start_date_buffer = start_date_buffer.strftime(date_fmt)
start_date_buffer

In [ ]:
class StockBacktestData:
  def __init__(self, ticker, start_date, end_date):
    self._ticker = ticker
    self._backtest_start_buffer_days = 365
    self._buffer_days = 90

    init_start_date, init_end_date = self._get_buffer_start_end_dates(start_date, end_date)
    self._data = self._download_stock_backtest_data(self._ticker, init_start_date, init_end_date)

  
  def _get_buffer_start_end_dates(self, start_date, end_date):
    date_fmt = '%Y-%m-%d'
    init_start_date = datetime.strptime(start_date, date_fmt) - timedelta(
        days=(self._backtest_start_buffer_days + self._buffer_days)
        )
    
    init_start_date = init_start_date.strftime(date_fmt)

    init_end_date = datetime.strptime(end_date, date_fmt) + timedelta(days=self._buffer_days)

    if init_end_date > datetime.today():
      init_end_date = datetime.today()

    init_end_date = init_end_date.strftime(date_fmt)

    return init_start_date, init_end_date


  def _get_backtest_start_date(self, start_date):
    date_fmt = '%Y-%m-%d'
    start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(
        days=self._backtest_start_buffer_days
        )
    
    start_date_buffer = start_date_buffer.strftime(date_fmt)
    return start_date_buffer


  def _download_stock_backtest_data(self, ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    return df


  def get_stock_backtest_data(self, start_date, end_date):
    start_date_buffer = self._get_backtest_start_date(start_date)
    df = self._data[(self._data.index >= start_date_buffer) & (self._data.index <= end_date)]
    return df.copy()

**Strategies**

In [ ]:
def strategy_KeltnerChannel_origin(df, **kwargs):
  n = kwargs.get('n', 10)
  data = df.copy()

  k_band = ta.volatility.KeltnerChannel(data.High, data.Low, data.Close, n)

  data['K_BAND_UB'] = k_band.keltner_channel_hband().round(4)
  data['K_BAND_LB'] = k_band.keltner_channel_lband().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)
  
  data['LONG'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)
  data['EXIT_LONG'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)

  data['SHORT'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)
  data['EXIT_SHORT'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_KeltnerChannel_origin_long(df, **kwargs):
  n = kwargs.get('n', 10)
  data = df.copy()

  k_band = ta.volatility.KeltnerChannel(data.High, data.Low, data.Close, n)

  data['K_BAND_UB'] = k_band.keltner_channel_hband().round(4)
  data['K_BAND_LB'] = k_band.keltner_channel_lband().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)
  
  data['LONG'] = (data.Close <= data.K_BAND_LB) & (data.CLOSE_PREV > data.K_BAND_LB)
  data['EXIT_LONG'] = (data.Close >= data.K_BAND_UB) & (data.CLOSE_PREV < data.K_BAND_UB)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_BollingerBands(df, **kwargs):
  n = kwargs.get('n', 10)
  n_rng = kwargs.get('n_rng', 2)
  data = df.copy()

  boll = ta.volatility.BollingerBands(data.Close, n, n_rng)

  data['BOLL_LBAND_INDI'] = boll.bollinger_lband_indicator()
  data['BOLL_UBAND_INDI'] = boll.bollinger_hband_indicator()

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = data.BOLL_LBAND_INDI == 1
  data['EXIT_LONG'] = data.BOLL_UBAND_INDI == 1

  data['SHORT'] = data.BOLL_UBAND_INDI == 1
  data['EXIT_SHORT'] = data.BOLL_LBAND_INDI == 1

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_BollingerBands_long(df, **kwargs):
  n = kwargs.get('n', 10)
  n_rng = kwargs.get('n_rng', 2)
  data = df.copy()
  
  boll = ta.volatility.BollingerBands(data.Close, n, n_rng)

  data['BOLL_LBAND_INDI'] = boll.bollinger_lband_indicator()
  data['BOLL_UBAND_INDI'] = boll.bollinger_hband_indicator()

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = data.BOLL_LBAND_INDI == 1
  data['EXIT_LONG'] = data.BOLL_UBAND_INDI == 1

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_MA(df, **kwargs):
  n = kwargs.get('n', 50)
  ma_type = kwargs.get('ma_type', 'sma')
  ma_type = ma_type.strip().lower()
  data = df.copy()
  
  if ma_type == 'sma':
    sma = ta.trend.SMAIndicator(data.Close, n)
    data['MA'] = sma.sma_indicator().round(4)
  elif ma_type == 'ema':
    ema = ta.trend.EMAIndicator(data.Close, n)
    data['MA'] = ema.ema_indicator().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)
  data['EXIT_LONG'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)

  data['SHORT'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)
  data['EXIT_SHORT'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

def strategy_MACrossover(df, **kwargs):
  n_slow = kwargs.get('n_slow', 13)
  n_fast = kwargs.get('n_fast', 5)
  n_middle = kwargs.get('n_middle', 8)
  data = df.copy()
  
  
  sma_fast = ta.trend.SMAIndicator(data.Close, n_fast)
  data['SMA_FAST'] = sma_fast.sma_indicator().round(4)
  sma_slow = ta.trend.SMAIndicator(data.Close, n_slow)
  data['SMA_SLOW'] = sma_slow.sma_indicator().round(4)
  sma_middle = ta.trend.SMAIndicator(data.Close, n_middle)
  data['SMA_MIDDLE'] = sma_middle.sma_indicator().round(4)

  data['SMA_FAST_PREV'] = data.SMA_FAST.shift(1)
  data['SMA_MIDDLE_PREV'] = data.SMA_MIDDLE.shift(1)
  data['SMA_SLOW_PREV'] = data.SMA_SLOW.shift(1)

  data['LONG'] = (data.SMA_FAST > data.SMA_MIDDLE) & (data.SMA_FAST > data.SMA_SLOW) & (data.SMA_MIDDLE > data.SMA_SLOW) & ((data.SMA_FAST_PREV < data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV < data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV < data.SMA_SLOW_PREV))
  data['EXIT_LONG'] = (data.SMA_FAST < data.SMA_MIDDLE) & (data.SMA_FAST < data.SMA_SLOW) & (data.SMA_MIDDLE < data.SMA_SLOW) & ((data.SMA_FAST_PREV > data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV > data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV > data.SMA_SLOW_PREV))

  data['SHORT'] = (data.SMA_FAST < data.SMA_MIDDLE) & (data.SMA_FAST < data.SMA_SLOW) & (data.SMA_MIDDLE < data.SMA_SLOW) & ((data.SMA_FAST_PREV > data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV > data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV > data.SMA_SLOW_PREV))
  data['EXIT_SHORT'] = (data.SMA_FAST > data.SMA_MIDDLE) & (data.SMA_FAST > data.SMA_SLOW) & (data.SMA_MIDDLE > data.SMA_SLOW) & ((data.SMA_FAST_PREV < data.SMA_MIDDLE_PREV) | (data.SMA_FAST_PREV < data.SMA_SLOW_PREV) | (data.SMA_MIDDLE_PREV < data.SMA_SLOW_PREV))

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_MA_long(df, **kwargs):
  n = kwargs.get('n', 50)
  ma_type = kwargs.get('ma_type', 'sma')
  ma_type = ma_type.strip().lower()
  data = df.copy()
  
  if ma_type == 'sma':
    sma = ta.trend.SMAIndicator(data.Close, n)
    data['MA'] = sma.sma_indicator().round(4)
  elif ma_type == 'ema':
    ema = ta.trend.EMAIndicator(data.Close, n)
    data['MA'] = ema.ema_indicator().round(4)

  data['CLOSE_PREV'] = data.Close.shift(1)

  data['LONG'] = (data.Close > data.MA) & (data.CLOSE_PREV <= data.MA)
  data['EXIT_LONG'] = (data.Close < data.MA) & (data.CLOSE_PREV >= data.MA)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_MACD(df, **kwargs):
  n_slow = kwargs.get('n_slow', 26)
  n_fast = kwargs.get('n_fast', 12)
  n_sign = kwargs.get('n_sign', 9)
  data = df.copy()

  macd = ta.trend.MACD(data.Close, n_slow, n_fast, n_sign)

  data['MACD_DIFF'] = macd.macd_diff().round(4)
  data['MACD_DIFF_PREV'] = data.MACD_DIFF.shift(1)

  data['LONG'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)

  data['SHORT'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(1)
  data.EXIT_LONG = data.EXIT_LONG.shift(1)
  data.SHORT = data.SHORT.shift(1)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(1)

  return data


def strategy_MACD_long(df, **kwargs):
  n_slow = kwargs.get('n_slow', 26)
  n_fast = kwargs.get('n_fast', 12)
  n_sign = kwargs.get('n_sign', 9)
  data = df.copy()

  macd = ta.trend.MACD(data.Close, n_slow, n_fast, n_sign)

  data['MACD_DIFF'] = macd.macd_diff().round(4)
  data['MACD_DIFF_PREV'] = data.MACD_DIFF.shift(1)

  data['LONG'] = (data.MACD_DIFF > 0) & (data.MACD_DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.MACD_DIFF < 0) & (data.MACD_DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_RSI(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  rsi = ta.momentum.RSIIndicator(data.Close, n)

  data['RSI'] = rsi.rsi().round(4)
  data['RSI_PREV'] = data.RSI.shift(1)

  data['LONG'] = (data.RSI > 30) & (data.RSI_PREV <= 30)
  data['EXIT_LONG'] = (data.RSI < 70) & (data.RSI_PREV >= 70)

  data['SHORT'] = (data.RSI < 70) & (data.RSI_PREV >= 70)
  data['EXIT_SHORT'] = (data.RSI > 30) & (data.RSI_PREV <= 30)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_RSI_long(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  rsi = ta.momentum.RSIIndicator(data.Close, n)

  data['RSI'] = rsi.rsi().round(4)
  data['RSI_PREV'] = data.RSI.shift(1)

  data['LONG'] = (data.RSI > 30) & (data.RSI_PREV <= 30)
  data['EXIT_LONG'] = (data.RSI < 70) & (data.RSI_PREV >= 70)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_WR(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  wr = ta.momentum.WilliamsRIndicator(data.High, data.Low, data.Close, n)

  data['WR'] = wr.williams_r().round(4)
  data['WR_PREV'] = data.WR.shift(1)

  data['LONG'] = (data.WR > -80) & (data.WR_PREV <= -80)
  data['EXIT_LONG'] = (data.WR < -20) & (data.WR_PREV >= -20)

  data['SHORT'] = (data.WR < -20) & (data.WR_PREV >= -20)
  data['EXIT_SHORT'] = (data.WR > -80) & (data.WR_PREV <= -80)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_WR_long(df, **kwargs):
  n = kwargs.get('n', 14)
  data = df.copy()

  wr = ta.momentum.WilliamsRIndicator(data.High, data.Low, data.Close, n)

  data['WR'] = wr.williams_r().round(4)
  data['WR_PREV'] = data.WR.shift(1)

  data['LONG'] = (data.WR > -80) & (data.WR_PREV <= -80)
  data['EXIT_LONG'] = (data.WR < -20) & (data.WR_PREV >= -20)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_Stochastic_fast(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  data['DIFF'] = data['K'] - data['D']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Stochastic_fast_long(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  data['DIFF'] = data['K'] - data['D']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def strategy_Stochastic_slow(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  dd = kwargs.get('dd', 3)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  
  ma = ta.trend.SMAIndicator(data.D, dd)
  data['DD'] = ma.sma_indicator().round(4)

  data['DIFF'] = data['D'] - data['DD']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Stochastic_slow_long(df, **kwargs):
  k = kwargs.get('k', 20)
  d = kwargs.get('d', 5)
  dd = kwargs.get('dd', 3)
  data = df.copy()

  sto = ta.momentum.StochasticOscillator(data.High, data.Low, data.Close, k, d)

  data['K'] = sto.stoch().round(4)
  data['D'] = sto.stoch_signal().round(4)
  
  ma = ta.trend.SMAIndicator(data.D, dd)
  data['DD'] = ma.sma_indicator().round(4)

  data['DIFF'] = data['D'] - data['DD']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)
  return data

In [ ]:
def strategy_Ichmoku(df, **kwargs):
  n_conv = kwargs.get('n_conv', 9)
  n_base = kwargs.get('n_base', 26)
  n_span_b = kwargs.get('n_span_b', 26)
  data = df.copy()
  
  ichmoku = ta.trend.IchimokuIndicator(data.High, data.Low, n_conv, n_base, n_span_b)

  data['BASE'] = ichmoku.ichimoku_base_line().round(4)
  data['CONV'] = ichmoku.ichimoku_conversion_line().round(4)

  data['DIFF'] = data['CONV'] - data['BASE']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)
  data['EXIT_SHORT'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data


def strategy_Ichmoku_long(df, **kwargs):
  n_conv = kwargs.get('n_conv', 9)
  n_base = kwargs.get('n_base', 26)
  n_span_b = kwargs.get('n_span_b', 26)
  data = df.copy()

  ichmoku = ta.trend.IchimokuIndicator(data.High, data.Low, n_conv, n_base, n_span_b)

  data['BASE'] = ichmoku.ichimoku_base_line().round(4)
  data['CONV'] = ichmoku.ichimoku_conversion_line().round(4)

  data['DIFF'] = data['CONV'] - data['BASE']
  data['DIFF_PREV'] = data.DIFF.shift(1)
  
  data['LONG'] = (data.DIFF > 0) & (data.DIFF_PREV <= 0)
  data['EXIT_LONG'] = (data.DIFF < 0) & (data.DIFF_PREV >= 0)

  data['SHORT'] = False
  data['EXIT_SHORT'] = False

  data.LONG = data.LONG.shift(0)
  data.EXIT_LONG = data.EXIT_LONG.shift(0)
  data.SHORT = data.SHORT.shift(0)
  data.EXIT_SHORT = data.EXIT_SHORT.shift(0)

  return data

In [ ]:
def get_stock_backtest_data(ticker, start_date, end_date):
  date_fmt = '%Y-%m-%d'

  start_date_buffer = datetime.strptime(start_date, date_fmt) - timedelta(days=365)
  start_date_buffer = start_date_buffer.strftime(date_fmt)

  df = yf.download(ticker, start=start_date_buffer, end=end_date)

  return df

In [ ]:
def prepare_stock_ta_backtest_data(df, start_date, end_date, strategy, **strategy_params):
  df_strategy = strategy(df, **strategy_params)
  bt_df = df_strategy[(df_strategy.index >= start_date) & (df_strategy.index <= end_date)]
  return bt_df

In [ ]:
def run_stock_ta_backtest(bt_df, stop_loss_lvl=None, etf_type=None, benchmark=None, strategy=None, params=None):
  balance = init_balance
  inc_balance = init_balance
  market_value = init_balance
  inc_market_value = init_balance
  pnl = 0.00
  position = 0
  inc_position = 0
  stopthresh = 'no'
  last_signal = 'hold'
  last_price = 0.00
  c = 0

  trade_date_start = []
  trade_date_end = []
  trade_days = []
  trade_side = []
  trade_pnl = []
  trade_ret = []
  trade_balance = []
  trade_stop = []
  trade_type = []
  trade_benchmark = []
  trade_inc_ret = []
  trade_strategy = []
  trade_params = []

  cum_value = []  
  cum_incValue = []

  

  for index, row in bt_df.iterrows():      
      # check and close any positions
      if row.EXIT_LONG and last_signal == 'long' and position > 0:
        trade_date_end.append(row.name)
        trade_days.append(c)

        pnl = (row.Close - last_price) * position
        trade_pnl.append(pnl)
        trade_ret.append((row.Close / last_price - 1) * 100)
        
        if row.name >= datetime.strptime('2020-11-9 00:00:00','%Y-%m-%d %H:%M:%S') :
          trade_inc_ret.append((row.Close / last_price - 1) * 100)
          inc_balance = inc_balance + (row.Close * inc_position)
        else:
          trade_inc_ret.append(0)

        balance = balance + (row.Close * position)
        trade_balance.append(balance)
        trade_stop.append(stop_loss_lvl)
        trade_type.append(etf_type)
        trade_benchmark.append(benchmark)
        trade_strategy.append(strategy)
        trade_params.append(params)
        
        
        pnl = 0
        position = 0
        last_signal = 'hold'

        c = 0

      # check signal and enter any possible position
      if row.LONG and last_signal != 'long':
        last_signal = 'long'
        last_price = row.Close
        trade_date_start.append(row.name)
        trade_side.append('long')

        position = int(balance / row.Close)
        cost = position * row.Close
        balance = balance - cost

        if row.name >= datetime.strptime('2020-11-9 00:00:00','%Y-%m-%d %H:%M:%S') :
          inc_position = int(inc_balance / row.Close)
          inc_cost = inc_position * row.Close
          inc_balance = inc_balance - inc_cost
        c = 0

      if row.SHORT and last_signal != 'short':
        last_signal = 'hold'
        
        c = 0
      
      if stop_loss_lvl:
        # check stop loss
        if position > 0 and ((row.Low / last_price)- 1) * 100 <= stop_loss_lvl and row.LONG == False:
          c = c + 1

          trade_date_end.append(row.name)
          trade_days.append(c)

          stop_loss_price = last_price + round(last_price * (stop_loss_lvl / 100), 4)

          pnl = (stop_loss_price - last_price) * position
          trade_pnl.append(pnl)
          trade_ret.append(((stop_loss_price / last_price) - 1) * 100)

          if row.name >= datetime.strptime('2020-11-9 00:00:00','%Y-%m-%d %H:%M:%S') :
            trade_inc_ret.append((row.Close / last_price - 1) * 100)
          else:
            trade_inc_ret.append(0)

          balance = balance + (stop_loss_price * position)

          if row.name >= datetime.strptime('2020-11-9 00:00:00','%Y-%m-%d %H:%M:%S') :
            inc_balance = inc_balance + (stop_loss_price * inc_position)

          trade_balance.append(balance)
          trade_stop.append(stop_loss_lvl)
          trade_type.append(etf_type)
          trade_benchmark.append(benchmark)
          trade_strategy.append(strategy)
          trade_params.append(params)
          
          pnl = 0
          position = 0
          inc_position = 0
          last_signal = 'hold'

          c = 0
    
      # compute market value and count days for any possible poisition
      if last_signal == 'hold':
        market_value = balance
        inc_market_value = inc_balance
      elif last_signal == 'long':
        c = c + 1
        market_value = (position * row.Close) + balance
        if row.name >= datetime.strptime('2020-11-9 00:00:00','%Y-%m-%d %H:%M:%S') :
          inc_market_value = (inc_position * row.Close) + inc_balance
      else: 
        c = c + 1
        #market_value = (row.Close - last_price) * position + balance
        #market_value = balance
      
      cum_value.append(market_value)
      cum_incValue.append(inc_market_value)


  # generate analysis
  # performance over time
  cum_ret_df = pd.DataFrame(cum_value, index=bt_df.index, columns=['CUM_RET'])
  cum_ret_df['CUM_RET'] = (market_value / init_balance - 1) * 100
  cum_ret_df['BUY_HOLD'] = (bt_df.Close / bt_df.Open.iloc[0] - 1) * 100
  cum_ret_df['ZERO'] = 0
  cum_ret_df['Market_Value'] = market_value
  cum_ret_df['INC_CUM_RET'] = (inc_market_value / init_balance - 1) * 100

  # trade stats
  size = min(len(trade_date_start), len(trade_date_end))

  tarde_dict = {
      'START': trade_date_start[:size],
      'END': trade_date_end[:size],
      'SIDE': trade_side[:size],
      'DAYS': trade_days[:size],
      'PNL': trade_pnl[:size],
      'RET': trade_ret[:size],
      'BAL': trade_balance[:size],
      'STOP':trade_stop[:size],
      'TYPE':trade_type[:size],
      'BENCH':trade_benchmark[:size],
      'INC_RET':trade_inc_ret[:size],
      'STRATEGY':trade_strategy[:size],
      'PARAMS':trade_params[:size]
  }

  trade_df = pd.DataFrame(tarde_dict)

  num_trades = trade_df.groupby('SIDE').count()[['START']]
  num_trades_win = trade_df[trade_df.PNL > 0].groupby('SIDE').count()[['START']]

  avg_days = trade_df.groupby('SIDE').mean()[['DAYS']]

  avg_ret = trade_df.groupby('SIDE').mean()[['RET']]
  avg_ret_win = trade_df[trade_df.PNL > 0].groupby('SIDE').mean()[['RET']]
  avg_ret_loss = trade_df[trade_df.PNL < 0].groupby('SIDE').mean()[['RET']]

  std_ret = trade_df.groupby('SIDE').std()[['RET']]

  detail_df = pd.concat([
                        num_trades, num_trades_win, avg_days,
                        avg_ret, avg_ret_win, avg_ret_loss, std_ret
                        ], axis=1, sort=False)

  detail_df.columns = [
                      'NUM_TRADES', 'NUM_TRADES_WIN', 'AVG_DAYS', 
                      'AVG_RET', 'AVG_RET_WIN', 'AVG_RET_LOSS', 'STD_RET'
                      ]

  detail_df.round(2)

  # max drawdown
  mv_df = pd.DataFrame(cum_value, index=bt_df.index, columns=['MV'])

  days = len(mv_df)

  roll_max = mv_df.MV.rolling(window=days, min_periods=1).max()
  drawdown_val = mv_df.MV - roll_max
  drawdown_pct = (mv_df.MV / roll_max - 1) * 100

  # return all stats
  return {
      'cum_ret_df': cum_ret_df,
      'max_drawdown': {
          'value': round(drawdown_val.min(), 0), 
          'pct': round(drawdown_pct.min(), 2)
          },
      'trade_stats': detail_df,
      'trade_details': trade_df
  }

In [ ]:
from pandas import ExcelWriter

writer = ExcelWriter("DetailedSignals:" + str(end_date) + ".xlsx")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

for elt_id, elt in enumerate(etflist):
  ticker = elt

  if ticker == 'sdow':
    benchmark = 'dia'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_WR'
    _n=7
    params = "(n=" + str(_n) + ")"

  if ticker == 'udow':
    benchmark = 'dia'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_Stochastic_slow'
    _k=17 
    _d=7
    _dd=3
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  if ticker == 'edz':
    benchmark = 'eem'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'edc':
    benchmark = 'eem'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'jpnl':
    benchmark = 'ewj'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 2
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'bzq':
    benchmark = 'ewz'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 20
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'brzu':
    benchmark = 'ewz'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 20
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'fngd':
    benchmark = 'dia'
    etf_type = 'bear'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 2
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'fngu':
    benchmark = 'fang'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_MACrossover'
    _n_slow=11 
    _n_fast=3 
    _n_middle=7
    params = "(n_slow=" + str(_n_slow) + ", n_fast=" + str(_n_fast) + ", n_middle=" + str(_n_middle) + ")"

  if ticker == 'yang':
    benchmark = 'fxi'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'yinn':
    benchmark = 'fxi'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'dust':
    benchmark = 'gdx'
    etf_type = 'bear'
    stop_loss_lvl = -5
    strategy = 'strategy_WR'
    _n=17
    params = "(n=" + str(_n) + ")"

  if ticker == 'nugt':
    benchmark = 'gdx'
    etf_type = 'bull'
    stop_loss_lvl = -5
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'jdst':
    benchmark = 'gdxj'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'jnug':
    benchmark = 'gdxj'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'gll':
    benchmark = 'gld'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'ugl':
    benchmark = 'gld'
    etf_type = 'bull'
    stop_loss_lvl = -4
    strategy = 'strategy_RSI'
    _n=11
    params = "(n=" + str(_n) + ")"

  if ticker == 'tza':
    benchmark = 'iwm'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_Stochastic_slow'
    _k=25 
    _d=10
    _dd=5
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  if ticker == 'tna':
    benchmark = 'iwm'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_Stochastic_slow'
    _k=23 
    _d=7
    _dd=5
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  if ticker == 'qid':
    benchmark = 'qqq'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_WR'
    _n=11
    params = "(n=" + str(_n) + ")"

  if ticker == 'qld':
    benchmark = 'qqq'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_Stochastic_slow'
    _k=18 
    _d=5
    _dd=4
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"


  if ticker == 'russ':
    benchmark = 'rsx'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'rusl':
    benchmark = 'rsx'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'drv':
    benchmark = 'rwr'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 2
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'drn':
    benchmark = 'rwr'
    etf_type = 'bull'
    stop_loss_lvl = -5
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"
    
  if ticker == 'zsl':
    benchmark = 'slv'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'agq':
    benchmark = 'slv'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_Stochastic_slow'
    _k=16 
    _d=9
    _dd=3
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  if ticker == 'soxs':
    benchmark = 'smh'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_MACD'
    _n_slow=15 
    _n_fast=22 
    _n_sign=7
    params = "(n_slow=" + str(_n_slow) + ", n_fast=" + str(_n_fast) + ", n_sign=" + str(_n_sign) + ")"

  if ticker == 'soxl':
    benchmark = 'smh'
    etf_type = 'bull'
    stop_loss_lvl = -5
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'spxs':
    benchmark = 'spy'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'spxl':
    benchmark = 'spy'
    etf_type = 'bull'
    stop_loss_lvl = -4
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'tmv':
    benchmark = 'tlt'
    etf_type = 'bear'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'tmf':
    benchmark = 'tlt'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'dwt':
    benchmark = 'uco'
    etf_type = 'bear'
    stop_loss_lvl = -4
    strategy = 'strategy_Stochastic_slow'
    _k=17 
    _d=9
    _dd=2
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  if ticker == 'uwt':
    benchmark = 'uco'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'kold':
    benchmark = 'ung'
    etf_type = 'bear'
    stop_loss_lvl = -5
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'boil':
    benchmark = 'ung'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'svxy':
    benchmark = 'vxxb'
    etf_type = 'bear'
    stop_loss_lvl = -4
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 2
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'uvxy':
    benchmark = 'vxxb'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 20
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'labd':
    benchmark = 'xbi'
    etf_type = 'bear'
    stop_loss_lvl = -4
    strategy = 'strategy_WR'
    _n=14
    params = "(n=" + str(_n) + ")"

  if ticker == 'labu':
    benchmark = 'xbi'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 10
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'bis':
    benchmark = 'xbi'
    etf_type = 'bear'
    stop_loss_lvl = -3
    strategy = 'strategy_WR'
    _n=19
    params = "(n=" + str(_n) + ")"

  if ticker == 'bib':
    benchmark = 'xbi'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'ery':
    benchmark = 'xle'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_Ichmoku'
    _n_conv=9 
    _n_base=24
    _n_span_b=26
    params = "(n_conv=" + str(_n_conv) + ", n_base=" + str(_n_base) + ", n_span_b=" + str(_n_span_b) + ")"
    
  if ticker == 'erx':
    benchmark = 'xle'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'faz':
    benchmark = 'xlf'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'fas':
    benchmark = 'xlf'
    etf_type = 'bull'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 25
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"


  if ticker == 'tecs':
    benchmark = 'xlk'
    etf_type = 'bear'
    stop_loss_lvl = -2
    strategy = 'strategy_BollingerBands'
    _n = 30
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'tecl':
    benchmark = 'xlk'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_BollingerBands'
    _n = 15
    _n_rng = 1
    params = "(n=" + str(_n) + ", n_rng=" + str(_n_rng) + ")"

  if ticker == 'jo':
    benchmark = 'jo'
    etf_type = 'bull'
    stop_loss_lvl = -3
    strategy = 'strategy_Stochastic_slow'
    _k=15 
    _d=6
    _dd=5
    params = "(k=" + str(_k) + ", d=" + str(_d) + ", dd=" + str(_dd) + ")"

  #Get the OHLC and Volume
  df = yf.download(ticker, start=start_date_buffer, end=end_date)

  df = get_stock_backtest_data(ticker, start_date, end_date)

  bt_df = df[(df.index >= start_date) & (df.index <= end_date)]

  if strategy == 'strategy_KeltnerChannel_origin':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_KeltnerChannel_origin, n=_n
      )

  if strategy == 'strategy_KeltnerChannel_origin_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_KeltnerChannel_origin_long, n=_n
      )

  if strategy == 'strategy_BollingerBands':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_BollingerBands, n=_n, n_rng=_n_rng
      )

  if strategy == 'strategy_BollingerBands_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_BollingerBands_long, n=_n, n_rng=_n_rng
      )

  if strategy == 'strategy_MA':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_MA, n=_n, ma_type=_ma_type
      )
    
  if strategy == 'strategy_MACrossover':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_MACrossover, n_slow=_n_slow, n_fast=_n_fast, n_middle=_n_middle
      )

  if strategy == 'strategy_MA_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_MA_long, n=_n, ma_type=_ma_type
      )

  if strategy == 'strategy_MACD':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_MACD, n_slow=_n_slow, n_fast=_n_fast, n_sign=_n_sign
      )

  if strategy == 'strategy_MACD_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_MACD_long, n_slow=_n_slow, n_fast=_n_fast, n_sign=_n_sign
      )

  if strategy == 'strategy_RSI':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_RSI, n=_n
      )

  if strategy == 'strategy_RSI_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_RSI_long, n=_n
      )

  if strategy == 'strategy_WR':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_WR, n=_n
      )
    bt_df.head()

  if strategy == 'strategy_WR_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_WR_long, n=_n
      )
    bt_df.head()

  if strategy == 'strategy_Stochastic_fast':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Stochastic_fast, k=_k, d=_d
      )

  if strategy == 'strategy_Stochastic_fast_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Stochastic_fast_long, k=_k, d=_d
      )

  if strategy == 'strategy_Stochastic_slow':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Stochastic_slow, k=_k, d=_d, dd=_dd
      )

  if strategy == 'strategy_Stochastic_slow_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Stochastic_slow_long, k=_k, d=_d, dd=_dd
      )

  if strategy == 'strategy_Ichmoku':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Ichmoku, n_conv=_n_conv, n_base=_n_base, n_span_b=_n_span_b
      )
    bt_df.head()

  if strategy == 'strategy_Ichmoku_long':
    bt_df = prepare_stock_ta_backtest_data(
      df, start_date, end_date, strategy_Ichmoku_long, n_conv=_n_conv, n_base=_n_base, n_span_b=_n_span_b
      )

  print(ticker)
  bt_df.to_excel(writer, ticker)
  writer.save()
  result = run_stock_ta_backtest(bt_df, stop_loss_lvl, etf_type, benchmark, strategy, params)
  df_details = result['trade_details']
  df_details.to_excel(writer, ticker + '_det')
  writer.save()
  
  cum_ret_df = result['cum_ret_df']
  cum_ret_df.to_excel(writer, ticker + '_cum')
  writer.save()
  print(cum_ret_df.tail(1))
  print('End ' + ticker)
  print('')

In [ ]:
toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))